# Region of Ineterst Time-Series

This notebook demonstrates generation and downloading ROI time-series. We will upload a region of interest and download time series from this ROI.

Reference information can be found at [Data access - Requesting time-series](https://docs.vandersat.com/VanderSat_API_UserGuide.html#requesting-time-series)

Other API end point operation can be found at [REST API - Planet Planetary Variables products](https://maps.vandersat.com/api/v2/)

In [ ]:
import os
import requests

## Authenticating 

In [ ]:
# Username and API Key stored as an env variable
AUTH = (os.getenv("VDS_USER"), os.getenv("VDS_PASS"))

In [ ]:
BASE_URL = "https://maps.vandersat.com/api/v2"

## Requests example
First, we will check if our authentication is working. We want a response code of `200` from this API call.

In [ ]:
# Authenticate
session = requests.Session()
session.auth = AUTH

# Request
response = session.get(BASE_URL)
response

## ROI operation examples 
We can upload a polygon (ROI) to an account and receive product time-series. First, let's list all the current ROIS inside the account.

In [ ]:
ROIS_URL = os.path.join(BASE_URL, "rois")

In [ ]:
# GET ROIS list
response = session.get(ROIS_URL)
response.json()["rois"]

Good chance there is no ROIS in the account yet, so let's create one first.

In [ ]:
# Create geometry of ROI
roi_geojson = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          4.909772872924805,
          51.989637331792146
        ],
        [
          4.922990798950195,
          51.989637331792146
        ],
        [
          4.922990798950195,
          51.99328417001946
        ],
        [
          4.909772872924805,
          51.99328417001946
        ],
        [
          4.909772872924805,
          51.989637331792146
        ]
      ]
    ]
}

# Name & describe the ROI
roi = {
    "name": "my_first_roi",
    "description": "practice",
    "geojson": roi_geojson,
}

# You can post multiple ROIS
rois = {
    "rois": [roi]
}

In [ ]:
# POST the ROI
response = session.post(ROIS_URL, json=rois)
response.json()

We can find our new ROI in the account's ROIS list.

In [ ]:
# GET ROIS list
response = session.get(ROIS_URL)
roi_list = response.json()["rois"]

## Product operation examples
Now that we posted our ROI, our system will automatically calculate the ROI time-series for all products available in the account. The processing can take some time, which depends on the product's `date range` and `pixel size`.

We can list the products available in the account and see their information.

In [ ]:
PRODUCT_URL = os.path.join(BASE_URL, "products")

In [ ]:
response = session.get(PRODUCT_URL)
product_list = response.json()["products"]

We can get the product's ROI time-series on demand; we only need to define which `product_name` and which `roi_id` & `start_time` & `end_time` as parameters.

In [ ]:
product_name = product_list[0]["api_name"]  # e.g. 'TEFF-AMSR2-DESC_V4.0_1000'
roi_id = roi_list[0]["id"]  # e.g. '37999'
parameters = {
    "roi_id": roi_id,
    "start_time": "2012-07-25",
    "end_time": "2022-09-14",
    "provide_coverage": "true"
}
roi_id

In [ ]:
ROI_TS_URL = os.path.join(PRODUCT_URL, product_name, "roi-time-series-sync")
response = session.get(ROI_TS_URL, params=parameters)
response.content

The API responds the time-series text-csv format, which you can decode and read with, for instance, pandas. 

In [ ]:
import pandas as pd
import io

# Read response
df = pd.read_csv(
    io.StringIO(response.content.decode("utf-8")),
    index_col=0,
    parse_dates=True,
    dayfirst=True,
    comment="#",
)
df

In [ ]:
# Write to csv
fname = "~/Downloads/pv_time_series.csv"
df.to_csv(fname)